# Algorithm

### Import modules

In [70]:
import numpy as np
import random 
import time
import csv
from datetime import datetime
from datetime import timedelta
import pandas as pd
from statistics import mean
from loguru import logger
import uuid

### Creation of the pacing Class (Speed algorithm)

In [3]:
class Algo:
    def __init__(self, daily_budget, nb_hours_day):
        """Class constructor"""
        # Fixed attributes
        self.daily_budget = daily_budget
        self.target = daily_budget/(nb_hours_day*3600)
        # Impossible day to initialize the setup
        self.day = 0
    
    def buying_decision(self, ts, price):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :price: price of the BR
        """
        # TS de la br
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        # If we begin a new day, we reset variables
        if self.day != day:
            logger.info("Changing day: \n")
            self.remaining_budget = self.daily_budget
            self.BT = [self.target]
            self.speed = pd.DataFrame({'V':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.engaged_budget = 0
            self.spent_budget = 0
        self.day = day
        # Remaining time before the end of day
        end_day = datetime(year,month,day,19,40,0,0)
        remaining_time =  datetime.timestamp(end_day) - ts
        # Calculation of bt
        created_time = self.speed.index[-1] - timedelta(minutes=30)
        self.remaining_budget = self.daily_budget - (self.engaged_budget + self.spent_budget)
        try:
            bt = self.remaining_budget * ((1+1*self.speed.V[self.speed.index > created_time].mean()) / remaining_time) 
        except ZeroDivisionError:
            bt = 1
        if bt < 0:
            bt = 1
        self.BT.append(bt)
        # Calculation of vt
        vt = self.BT[-1] - self.BT[-2] 
        self.speed = self.speed.append(pd.DataFrame({'V':vt}, index=[datetime.fromtimestamp(ts)]))
        if (bt >= self.target or bt<=0) and (self.remaining_budget - price) >= 0:
            buying = True
            self.engaged_budget += price
        else:
            buying = False

        return (buying, bt, vt)     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            if ev['status'] == 'win':
                self.engaged_budget -= ev['br_price']
                self.spent_budget += ev['br_price']
            else:
                self.engaged_budget -= ev['br_price']

In [4]:
data = pd.read_csv('wd_10-07-2020_08-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']

In [5]:
pacing = Algo(daily_budget=3000, nb_hours_day=14)

In [6]:
buyings = list()
remaining = list()
bt = list()
target = list()
vt = list()
spent = list()
engaged = list()
pending_notifications = list()
day = 10
for current_ts, row in data.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        engaged[-1] = pacing.engaged_budget
        pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
        remaining[-1] = pacing.remaining_budget
        spent[-1] = pacing.spent_budget
    
    # Receive BR
    decision = pacing.buying_decision(row['timestamp'], row['price'])
    logging.info(f"{current_ts} -> sending BR")
    # Making a decision
    if decision[0]:
        #Buying
        logging.info(f"{current_ts} <- receiving buy")
        next_notif_ts = current_ts + timedelta(seconds=row['seconds_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['price'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    target.append(pacing.target)
    bt.append(decision[1])
    vt.append(decision[2])
    buyings.append(decision[0])
    remaining.append(pacing.remaining_budget)
    spent.append(pacing.spent_budget)
    engaged.append(pacing.engaged_budget)
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
engaged[-1] = pacing.engaged_budget
pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
remaining[-1] = pacing.remaining_budget
spent[-1] = pacing.spent_budget
data['target'] = target
data['bt'] = bt
data['vt'] = vt
data['buying'] = buyings
data['remaining_budget'] = remaining
data['engaged_budget'] = engaged
data['spent_budget'] = spent

In [7]:
data.groupby(data.index.day).tail(1)

,ID,weekday,timestamp,nb_imp,price,win,seconds_notif,target,bt,vt,buying,remaining_budget,engaged_budget,spent_budget
Date,,,,,,,,,,,,,,
2020-07-10 19:50:35,1653,4,1.594403e+09,2,2,True,117,0.059524,1.0,0.0,True,773,0,2227
2020-07-11 19:42:32,2759,5,1.594489e+09,5,5,True,144,0.059524,1.0,0.0,True,488,0,2512
2020-07-12 19:59:59,4768,6,1.594577e+09,4,4,True,288,0.059524,1.0,0.0,False,1,0,2999
2020-07-13 19:54:56,6522,0,1.594663e+09,2,2,True,684,0.059524,-0.0,0.0,False,0,0,3000
2020-07-14 19:54:27,8518,1,1.594749e+09,8,8,True,93,0.059524,-0.0,0.0,False,0,0,3000
2020-07-15 19:59:58,10123,2,1.594836e+09,0,0,True,429,0.059524,-0.0,0.0,True,0,0,3000
2020-07-16 19:59:59,12211,3,1.594922e+09,1,1,True,322,0.059524,-0.0,0.0,False,5,0,2995
2020-07-17 19:56:11,13782,4,1.595009e+09,3,3,True,704,0.059524,1.0,0.0,True,951,0,2049
2020-07-18 19:57:21,15361,5,1.595095e+09,0,0,False,33,0.059524,1.0,0.0,True,855,0,2145


In [8]:
print(f"In a 30 days average, the remaining budget at the end of the day is {data.remaining_budget.groupby(data.index.day).tail(1).mean()}")

In a 30 days average, the remaining budget at the end of the day is 197.33333333333334


In [9]:
data.to_csv('pacing(vt)_10-07-2020_08-08-2020.csv', index=True)

### Creation of the pacing Class (Acceleration algorithm)

In [10]:
class Algo:
    def __init__(self, daily_budget, nb_hours_day):
        """Class constructor"""
        # Fixed attributes
        self.daily_budget = daily_budget
        self.target = daily_budget/(nb_hours_day*3600)
        # Impossible day to initialize the setup
        self.day = 0
    
    def buying_decision(self, ts, price):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :price: price of the BR
        """
        # TS de la br
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        # If we begin a new day, we reset variables
        if self.day != day:
            logging.info("-\n CHANGING DAY \n")
            self.remaining_budget = self.daily_budget
            self.BT = [self.target]
            self.acceleration = pd.DataFrame({'A':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.speed = [0]
            self.engaged_budget = 0
            self.spent_budget = 0
        self.day = day
        # Remaining time before the end of the day
        end_day = datetime(year,month,day,19,40,0,0)
        remaining_time =  datetime.timestamp(end_day) - ts
        # Calculation of bt
        created_time = self.acceleration.index[-1] - timedelta(minutes=30)
        self.remaining_budget = self.daily_budget - (self.engaged_budget + self.spent_budget)
        try:
            bt = self.remaining_budget * ((1+1*self.acceleration.A[self.acceleration.index > created_time].mean()) / remaining_time) 
        except ZeroDivisionError:
            bt = 1
        if bt < 0:
            bt = 1
        self.BT.append(bt)
        # Calculation of vt
        vt = self.BT[-1] - self.BT[-2] 
        self.speed.append(vt)
        at = self.speed[-1] - self.speed[-2]
        self.acceleration = self.acceleration.append(pd.DataFrame({'A':at}, index=[datetime.fromtimestamp(ts)]))
        if (bt >= self.target or bt<=0) and (self.remaining_budget - price) >= 0:
            buying = True
            self.engaged_budget += price
        else:
            buying = False

        return (buying, bt, at)     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            logging.info(f"{ev['timestamp']} -> Sending {ev['status']} notif with ID {ev['id']}")
            if ev['status'] == 'win':
                self.engaged_budget -= ev['br_price']
                self.spent_budget += ev['br_price']
            else:
                self.engaged_budget -= ev['br_price']

In [11]:
data = pd.read_csv('wd_10-07-2020_08-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']

In [12]:
pacing = Algo(daily_budget=3000, nb_hours_day=14)

In [13]:
buyings = list()
remaining = list()
bt = list()
target = list()
at = list()
spent = list()
engaged = list()
pending_notifications = list()
day = 10
for current_ts, row in data.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        engaged[-1] = pacing.engaged_budget
        pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
        remaining[-1] = pacing.remaining_budget
        spent[-1] = pacing.spent_budget
    
    # Receive BR
    decision = pacing.buying_decision(row['timestamp'], row['price'])
    logging.info(f"{current_ts} -> sending BR")
    # Making a decision
    if decision[0]:
        #Buying
        logging.info(f"{current_ts} <- receiving buy")
        next_notif_ts = current_ts + timedelta(seconds=row['seconds_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        logging.info(f" | {current_ts} decided {status}, will be notified at {next_notif_ts} with ID {notif_id}")
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['price'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    target.append(pacing.target)
    bt.append(decision[1])
    at.append(decision[2])
    buyings.append(decision[0])
    remaining.append(pacing.remaining_budget)
    spent.append(pacing.spent_budget)
    engaged.append(pacing.engaged_budget)
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
engaged[-1] = pacing.engaged_budget
pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
remaining[-1] = pacing.remaining_budget
spent[-1] = pacing.spent_budget
data['target'] = target
data['bt'] = bt
data['at'] = at
data['buying'] = buyings
data['remaining_budget'] = remaining
data['engaged_budget'] = engaged
data['spent_budget'] = spent

In [14]:
data.groupby(data.index.day).tail(1)

,ID,weekday,timestamp,nb_imp,price,win,seconds_notif,target,bt,at,buying,remaining_budget,engaged_budget,spent_budget
Date,,,,,,,,,,,,,,
2020-07-10 19:50:35,1653,4,1.594403e+09,2,2,True,117,0.059524,1.0,0.0,True,773,0,2227
2020-07-11 19:42:32,2759,5,1.594489e+09,5,5,True,144,0.059524,1.0,0.0,True,488,0,2512
2020-07-12 19:59:59,4768,6,1.594577e+09,4,4,True,288,0.059524,1.0,0.0,False,1,0,2999
2020-07-13 19:54:56,6522,0,1.594663e+09,2,2,True,684,0.059524,-0.0,0.0,False,0,0,3000
2020-07-14 19:54:27,8518,1,1.594749e+09,8,8,True,93,0.059524,-0.0,0.0,False,0,0,3000
2020-07-15 19:59:58,10123,2,1.594836e+09,0,0,True,429,0.059524,-0.0,0.0,True,0,0,3000
2020-07-16 19:59:59,12211,3,1.594922e+09,1,1,True,322,0.059524,-0.0,0.0,False,5,0,2995
2020-07-17 19:56:11,13782,4,1.595009e+09,3,3,True,704,0.059524,1.0,0.0,True,951,0,2049
2020-07-18 19:57:21,15361,5,1.595095e+09,0,0,False,33,0.059524,1.0,0.0,True,855,0,2145


In [15]:
print(f"In a 30 days average, the remaining budget at the end of the day is {data.remaining_budget.groupby(data.index.day).tail(1).mean()}")

In a 30 days average, the remaining budget at the end of the day is 197.33333333333334


In [16]:
data.to_csv('pacing(at)_10-07-2020_08-08-2020.csv', index=True)

### Creation of the pacing Class (Acceleration + evolutive target algorithm)

In [115]:
class Algo:
    def __init__(self, daily_budget, nb_hours_day, prop_table):
        """Class constructor"""
        # Fixed attributes
        self.daily_budget = daily_budget
        self.prop_table = prop_table
        # Impossible day and hour to initialize the setup
        self.day = 0
        self.current_hour = 0
        self.remaining_budget_hour = 0 # Initialize before buying
    
    def buying_decision(self, ts, price):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :price: price of the BR
        """
        # TS de la BR
        weekday = datetime.fromtimestamp(ts).weekday()
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        hour = datetime.fromtimestamp(ts).hour
        # Changement of hour
        if hour != self.current_hour:
            self.current_hour = hour
            # Evolutive target
            self.budget_hour = (self.prop_table.loc[hour, str(weekday)]/100)*self.daily_budget + self.remaining_budget_hour
            self.target = self.budget_hour/3600
            self.spent_hour = 0
        # If we begin a new day, we reset variables
        if self.day != day:
            self.remaining_budget = self.daily_budget
            self.BT = [self.target]
            self.acceleration = pd.DataFrame({'A':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.speed = [0]
            self.engaged_budget = 0
            self.spent_budget = 0
        self.day = day
        # Remaining time before the end of the hour
        end_hour = datetime(year,month,day,hour+1,0,0,0)
        remaining_time =  datetime.timestamp(end_hour) - ts
        # Calculation of bt
        created_time = self.acceleration.index[-1] - timedelta(minutes=30)
        self.remaining_budget = self.daily_budget - (self.engaged_budget + self.spent_budget)
        self.remaining_budget_hour = self.budget_hour - self.spent_hour
        try:
            bt = self.remaining_budget_hour * ((1+1*self.acceleration.A[self.acceleration.index > created_time].mean()) / remaining_time) 
        except ZeroDivisionError:
            bt = 1
        self.BT.append(bt)
        # Calculation of vt
        vt = self.BT[-1] - self.BT[-2] 
        self.speed.append(vt)
        at = self.speed[-1] - self.speed[-2]
        self.acceleration = self.acceleration.append(pd.DataFrame({'A':at}, index=[datetime.fromtimestamp(ts)]))
        if (bt >= self.target) and (self.remaining_budget - price) >= 0:
            buying = True
            self.engaged_budget += price
            self.spent_hour += price
        else:
            buying = False

        return (buying, bt, at)     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            if ev['status'] == 'win':
                self.engaged_budget -= ev['br_price']
                self.spent_budget += ev['br_price']
            else:
                self.engaged_budget -= ev['br_price']
                self.spent_hour -= ev['br_price']

In [116]:
data = pd.read_csv('wd_10-07-2020_08-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']
prop = pd.read_csv('proportion_table.csv', index_col = 'hour')

In [117]:
pacing = Algo(daily_budget=3000, nb_hours_day=14, prop_table = prop)

In [118]:
records = list()
pending_notifications = list()
day = 10
for current_ts, row in data.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        records[-1]['engaged'] = pacing.engaged_budget
        pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
        records[-1]['remaining'] = pacing.remaining_budget
        records[-1]['spent'] = pacing.spent_budget
        
    # Receive BR
    buying, bt, at = pacing.buying_decision(row['timestamp'], row['price'])
    # Making a decision
    if buying:
        #Buying
        next_notif_ts = current_ts + timedelta(seconds=row['seconds_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['price'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    record = {
        'target':pacing.target,
        'bt':bt,
        'at':at,
        'buying':buying,
        'remaining':pacing.remaining_budget,
        'spent':pacing.spent_budget,
        'engaged':pacing.engaged_budget
    }
    records.append(record)
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
records[-1]['engaged'] = pacing.engaged_budget
pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
records[-1]['remaining'] = pacing.remaining_budget
records[-1]['spent'] = pacing.spent_budget
pacing_df = pd.DataFrame.from_records(records)
new_df = pd.concat([data.reset_index(),pacing_df], axis=1, ignore_index=True)
new_df.columns = ['Date','id','weekday','timestamp','nb_imp','price','win','seconds_notif',
                 'target','bt','at','buying','remaining','spent','engaged']
new_df.set_index('Date', inplace=True)

In [119]:
new_df.groupby(data.index.day).tail(1)

,id,weekday,timestamp,nb_imp,price,win,seconds_notif,target,bt,at,buying,remaining,spent,engaged
Date,,,,,,,,,,,,,,
2020-07-10 19:50:35,1653,4,1.594403e+09,2,2,True,117,0.100556,0.263677,-0.003114,True,145,2855,0
2020-07-11 19:42:32,2759,5,1.594489e+09,5,5,True,144,0.084167,0.158398,0.001220,False,0,3000,0
2020-07-12 19:59:59,4768,6,1.594577e+09,4,4,True,288,0.050000,204.720796,192.931425,False,0,3000,0
2020-07-13 19:54:56,6522,0,1.594663e+09,2,2,True,684,0.082778,0.444130,0.001489,False,0,3000,0
2020-07-14 19:54:27,8518,1,1.594749e+09,8,8,True,93,0.083333,0.525553,-0.002855,False,0,3000,0
2020-07-15 19:59:58,10123,2,1.594836e+09,0,0,True,429,0.075556,117.877895,69.011756,True,0,3000,0
2020-07-16 19:59:59,12211,3,1.594922e+09,1,1,True,322,0.075278,232.457346,139.072456,False,0,3000,0
2020-07-17 19:56:11,13782,4,1.595009e+09,3,3,True,704,0.100278,0.825547,-0.009897,False,0,3000,0
2020-07-18 19:57:21,15361,5,1.595095e+09,0,0,False,33,0.085833,0.906071,-0.004471,True,0,3000,0


In [121]:
print(f"In a 30 days average, the remaining budget at the end of the day is {new_df.remaining.groupby(data.index.day).tail(1).mean()}")

In a 30 days average, the remaining budget at the end of the day is 4.933333333333334


In [23]:
data.to_csv('pacing(at_and_evolutive)_10-07-2020_08-08-2020.csv', index=True)

### Class with both acceleration and speed

In [28]:
class Algo:
    def __init__(self, daily_budget, nb_hours_day):
        """Class constructor"""
        # Fixed attributes
        self.daily_budget = daily_budget
        self.target = daily_budget/(nb_hours_day*3600)
        # Impossible day to initialize the setup
        self.day = 0
    
    def buying_decision(self, ts, price):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :price: price of the BR
        """
        # TS de la br
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        # If we begin a new day, we reset variables
        if self.day != day:
            logging.info("-\n CHANGING DAY \n")
            self.remaining_budget = self.daily_budget
            self.BT = [self.target]
            self.acceleration = pd.DataFrame({'A':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.speed = pd.DataFrame({'V':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.engaged_budget = 0
            self.spent_budget = 0
        self.day = day
        # Remaining time before the end of the day
        end_day = datetime(year,month,day,19,40,0,0)
        remaining_time =  datetime.timestamp(end_day) - ts
        # Calculation of bt
        created_time_A = self.acceleration.index[-1] - timedelta(minutes=30)
        created_time_S = self.speed.index[-1] - timedelta(minutes=30)
        self.remaining_budget = self.daily_budget - (self.engaged_budget + self.spent_budget)
        alpha = 100 * (self.acceleration.A[self.acceleration.index > created_time_A].mean())
        try:
            bt = self.remaining_budget * ((1+alpha*self.speed.V[self.speed.index > created_time_S].mean()) / remaining_time) 
        except ZeroDivisionError:
            bt = 1
        if bt < 0:
            bt = 1
        self.BT.append(bt)
        # Calculation of vt
        vt = self.BT[-1] - self.BT[-2] 
        self.speed = self.speed.append(pd.DataFrame({'V':vt}, index=[datetime.fromtimestamp(ts)]))
        at = self.speed.V[-1] - self.speed.V[-2]
        self.acceleration = self.acceleration.append(pd.DataFrame({'A':at}, index=[datetime.fromtimestamp(ts)]))
        if (bt >= self.target or bt<=0) and (self.remaining_budget - price) >= 0:
            buying = True
            self.engaged_budget += price
        else:
            buying = False

        return (buying, bt)     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            logging.info(f"{ev['timestamp']} -> Sending {ev['status']} notif with ID {ev['id']}")
            if ev['status'] == 'win':
                self.engaged_budget -= ev['br_price']
                self.spent_budget += ev['br_price']
            else:
                self.engaged_budget -= ev['br_price']

In [29]:
data = pd.read_csv('wd_10-07-2020_08-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']

In [30]:
pacing = Algo(daily_budget=3000, nb_hours_day=14)

In [31]:
buyings = list()
remaining = list()
bt = list()
target = list()
spent = list()
engaged = list()
pending_notifications = list()
day = 10
for current_ts, row in data.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        engaged[-1] = pacing.engaged_budget
        pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
        remaining[-1] = pacing.remaining_budget
        spent[-1] = pacing.spent_budget
    
    # Receive BR
    decision = pacing.buying_decision(row['timestamp'], row['price'])
    logging.info(f"{current_ts} -> sending BR")
    # Making a decision
    if decision[0]:
        #Buying
        logging.info(f"{current_ts} <- receiving buy")
        next_notif_ts = current_ts + timedelta(seconds=row['seconds_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        logging.info(f" | {current_ts} decided {status}, will be notified at {next_notif_ts} with ID {notif_id}")
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['price'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    target.append(pacing.target)
    bt.append(decision[1])
    buyings.append(decision[0])
    remaining.append(pacing.remaining_budget)
    spent.append(pacing.spent_budget)
    engaged.append(pacing.engaged_budget)
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
engaged[-1] = pacing.engaged_budget
pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
remaining[-1] = pacing.remaining_budget
spent[-1] = pacing.spent_budget
data['target'] = target
data['bt'] = bt
data['buying'] = buyings
data['remaining_budget'] = remaining
data['engaged_budget'] = engaged
data['spent_budget'] = spent

In [32]:
data.remaining_budget.groupby(data.index.day).tail(1).mean()

197.33333333333334